In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn import preprocessing

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
X = pd.read_pickle('../../data/preprocessed/merged/mergedX.pkl')

In [3]:
# Apply label encoding to X
label_encoders = {}
string_columns = X.select_dtypes(include=['object'])
for column in string_columns:
    label_encoders[column] = preprocessing.LabelEncoder().fit(string_columns[column])
    X[column] = label_encoders[column].transform(string_columns[column])

In [4]:
# Load Y and merge with X
Y = pd.read_csv('../../data/preprocessed/merged/Y.csv')
Y = Y.merge(X, how='right', on='SEQN').drop(columns=[column for column in list(X.columns) if column != 'SEQN'])

In [5]:
Y = Y.drop(columns=['Unnamed: 0'])
Y = Y.applymap(lambda x: int(x))

In [6]:
y_cleaned = Y[Y.columns[1:]].as_matrix()
y_cleaned

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [7]:
scores_rf = {}
n_estimators = [10, 50, 100, 200, 500]
max_depths = [1, 10, 15, 30, 50]

for n_estimator in n_estimators:
    for max_depth in max_depths:
        rfc = RandomForestClassifier(n_estimators=n_estimator, max_depth=max_depth, random_state=0)
        scores_rf[str(n_estimator) + ' ' + str(max_depth)] = cross_val_score(rfc, X[X.columns[1:]].as_matrix(), Y[Y.columns[1:]].as_matrix(), cv=5, scoring='f1_samples')

for key in scores_rf:
    print("n_estimators - max_depths", key, " Fscore: %0.2f (+/- %0.2f)" % (scores_rf[key].mean(), scores_rf[key].std() * 2))

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

n_estimators - max_depths 10 1  Fscore: 0.04 (+/- 0.16)
n_estimators - max_depths 10 10  Fscore: 0.08 (+/- 0.09)
n_estimators - max_depths 10 15  Fscore: 0.10 (+/- 0.06)
n_estimators - max_depths 10 30  Fscore: 0.15 (+/- 0.03)
n_estimators - max_depths 10 50  Fscore: 0.17 (+/- 0.02)
n_estimators - max_depths 50 1  Fscore: 0.05 (+/- 0.19)
n_estimators - max_depths 50 10  Fscore: 0.07 (+/- 0.13)
n_estimators - max_depths 50 15  Fscore: 0.08 (+/- 0.10)
n_estimators - max_depths 50 30  Fscore: 0.12 (+/- 0.05)
n_estimators - max_depths 50 50  Fscore: 0.14 (+/- 0.03)
n_estimators - max_depths 100 1  Fscore: 0.05 (+/- 0.18)
n_estimators - max_depths 100 10  Fscore: 0.06 (+/- 0.13)
n_estimators - max_depths 100 15  Fscore: 0.07 (+/- 0.11)
n_estimators - max_depths 100 30  Fscore: 0.11 (+/- 0.07)
n_estimators - max_depths 100 50  Fscore: 0.14 (+/- 0.05)
n_estimators - max_depths 200 1  Fscore: 0.04 (+/- 0.16)
n_estimators - max_depths 200 10  Fscore: 0.06 (+/- 0.13)
n_estimators - max_depths 20

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [8]:
n_neighbors = [2, 5, 10, 15, 20, 30, 50, 100, 200, 500]
scores_knn = {}
for n_neighbor in n_neighbors:
    knn = KNeighborsClassifier(n_neighbors=n_neighbor)
    scores_knn[n_neighbor] = cross_val_score(knn, X[X.columns[1:]].as_matrix(), Y[Y.columns[1:]].as_matrix(), cv=10, scoring='f1_samples')
for key in scores_knn:
    print("n_neighbor: ", key, "Fscore: %0.2f (+/- %0.2f)" % (scores_knn[key].mean(), scores_knn[key].std() * 2))


/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

n_neighbor:  2 Fscore: 0.12 (+/- 0.02)
n_neighbor:  5 Fscore: 0.20 (+/- 0.03)
n_neighbor:  10 Fscore: 0.13 (+/- 0.02)
n_neighbor:  15 Fscore: 0.17 (+/- 0.02)
n_neighbor:  20 Fscore: 0.13 (+/- 0.02)
n_neighbor:  30 Fscore: 0.12 (+/- 0.02)
n_neighbor:  50 Fscore: 0.11 (+/- 0.02)
n_neighbor:  100 Fscore: 0.09 (+/- 0.03)
n_neighbor:  200 Fscore: 0.08 (+/- 0.04)
n_neighbor:  500 Fscore: 0.05 (+/- 0.07)


/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [9]:
columns_to_be_removed = []
for column in Y:
    if column != 'SEQN':
        if (Y[column].sum()/Y.shape[0]) < 0.01:
            columns_to_be_removed.append(column)
Y_less_sparsed = Y.drop(columns=columns_to_be_removed)
# for column in y_sum:
#     print(column)
#     print(y_sum[column][0])

In [10]:
scores_rf = {}
n_estimators = [10, 50, 100, 200, 500]
max_depths = [1, 10, 15, 30, 50]

for n_estimator in n_estimators:
    for max_depth in max_depths:
        rfc = RandomForestClassifier(n_estimators=n_estimator, max_depth=max_depth, random_state=0)
        scores_rf[str(n_estimator) + ' ' + str(max_depth)] = cross_val_score(rfc, X[X.columns[1:]].as_matrix(), Y_less_sparsed[Y_less_sparsed.columns[1:]].as_matrix(), cv=5, scoring='f1_samples')

for key in scores_rf:
    print(key, " Fscore: %0.2f (+/- %0.2f)" % (scores_rf[key].mean(), scores_rf[key].std() * 2))

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

10 1  Fscore: 0.04 (+/- 0.16)
10 10  Fscore: 0.09 (+/- 0.10)
10 15  Fscore: 0.11 (+/- 0.06)
10 30  Fscore: 0.15 (+/- 0.02)
10 50  Fscore: 0.18 (+/- 0.02)
50 1  Fscore: 0.05 (+/- 0.18)
50 10  Fscore: 0.07 (+/- 0.12)
50 15  Fscore: 0.08 (+/- 0.10)
50 30  Fscore: 0.12 (+/- 0.06)
50 50  Fscore: 0.14 (+/- 0.02)
100 1  Fscore: 0.05 (+/- 0.18)
100 10  Fscore: 0.06 (+/- 0.12)
100 15  Fscore: 0.08 (+/- 0.10)
100 30  Fscore: 0.11 (+/- 0.06)
100 50  Fscore: 0.14 (+/- 0.02)
200 1  Fscore: 0.04 (+/- 0.17)
200 10  Fscore: 0.06 (+/- 0.13)
200 15  Fscore: 0.07 (+/- 0.12)
200 30  Fscore: 0.10 (+/- 0.07)
200 50  Fscore: 0.13 (+/- 0.04)
500 1  Fscore: 0.04 (+/- 0.16)
500 10  Fscore: 0.05 (+/- 0.13)
500 15  Fscore: 0.06 (+/- 0.12)
500 30  Fscore: 0.09 (+/- 0.07)
500 50  Fscore: 0.12 (+/- 0.05)


/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [11]:
n_neighbors = [2, 5, 10, 15, 20, 30, 50, 100, 200, 500]
scores_knn = {}
for n_neighbor in n_neighbors:
    knn = KNeighborsClassifier(n_neighbors=n_neighbor)
    scores_knn[n_neighbor] = cross_val_score(knn, X[X.columns[1:]].as_matrix(), Y_less_sparsed[Y_less_sparsed.columns[1:]].as_matrix(), cv=10, scoring='f1_samples')
for key in scores_knn:
    print("n_neighbor: ", key, "Fscore: %0.2f (+/- %0.2f)" % (scores_knn[key].mean(), scores_knn[key].std() * 2))


/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/cl

n_neighbor:  2 Fscore: 0.12 (+/- 0.02)
n_neighbor:  5 Fscore: 0.20 (+/- 0.03)
n_neighbor:  10 Fscore: 0.13 (+/- 0.02)
n_neighbor:  15 Fscore: 0.17 (+/- 0.03)
n_neighbor:  20 Fscore: 0.13 (+/- 0.02)
n_neighbor:  30 Fscore: 0.12 (+/- 0.02)
n_neighbor:  50 Fscore: 0.11 (+/- 0.02)
n_neighbor:  100 Fscore: 0.09 (+/- 0.03)
n_neighbor:  200 Fscore: 0.08 (+/- 0.04)
n_neighbor:  500 Fscore: 0.05 (+/- 0.07)


/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/tri/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [12]:
# Cant converge
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.svm import SVC

# ovsr = OneVsRestClassifier(SVC(kernel='linear'), n_jobs=-1)
# scores_ovsr = cross_val_score(ovsr, X[X.columns[1:]].as_matrix(), Y[Y.columns[1:]].as_matrix(), cv=5, scoring='f1_samples')

# OneVsRestClassifier(SVC(kernel='linear'))
# print("Fscore: %0.2f (+/- %0.2f)" % (scores_ovsr[key].mean(), scores_ovsr[key].std() * 2))

In [13]:
# from sklearn.neighbors import RadiusNeighborsClassifier
# # n_neighbors = [2, 5, 10, 15, 30, 50, 100]
# scores_rnn = {}
# radiuses = [2.0]
# weights = ['uniform', 'distance'],
# algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
# rnn = RadiusNeighborsClassifier(radius = 100.0, weights='uniform')
# scores_rnn = cross_val_score(rnn, X[X.columns[1:]].as_matrix(), Y[Y.columns[1:]].as_matrix(), cv=10, scoring='f1_samples')
# print("Fscore: %0.2f (+/- %0.2f)" % (scores_rnn.mean(), scores_rnn[key].std() * 2))
# # for n_neighbor in n_neighbors:
# #     knn = KNeighborsClassifier(n_neighbors=n_neighbor)
# #     scores_knn[n_neighbor] = cross_val_score(knn, X[X.columns[1:]].as_matrix(), Y[Y.columns[1:]].as_matrix(), cv=10, scoring='f1_samples')
# # for key in scores_knn:
# #     print("n_neighbor: ", key, "Fscore: %0.2f (+/- %0.2f)" % (scores_knn[key].mean(), scores_knn[key].std() * 2))


In [18]:
print(Y.shape)
print(Y_less_sparsed.shape)

(3061, 82)
(3061, 51)


In [15]:
# knnc.predict(X[X.columns[1:]].iloc[[0]].as_matrix())

In [16]:
# Y[Y.columns[1:]].iloc[[0]].as_matrix()